# Frank and Goodman (2012)

This is a python implementation of the simple RSA model presented in [Frank and Goodman (2012)](https://science.sciencemag.org/content/336/6084/998).

In [216]:
import numpy as np
import pandas as pd

In [217]:
objects2 = ["blue square", "blue circle", "green square"]

In [218]:
utterances = ["blue", "green", "square", "circle"]

In [219]:
def uniform_probs():
    return pd.Series(np.ones(len(objects2))/len(objects2), index=objects2)

In [220]:
uniform_probs()

blue square     0.333333
blue circle     0.333333
green square    0.333333
dtype: float64

In [225]:
costs = {"blue": 1,
            "green": 1,
            "square": 1,
            "circle": 1}

def cost(utterance):
    return costs[utterance]

alpha = 1

## Meaning function
Returns true or false if the given object can be described by the given utterance.

In [221]:
def meaning(utterance, obj):
    return utterance in obj

## Literal listener
Returns a probability distribution over objects for a given utterance.

In [222]:
# need to return distribution over objects
def literal_listener(utterance):
    truth_values = np.array(list(map(lambda x:meaning(utterance, x), objects2)))
    return truth_values * uniform_probs() / pd.Series.sum(truth_values * uniform_probs())

In [223]:
def L1_wrapper():
    return pd.DataFrame({k:literal_listener(k) for k in utterances})

In [224]:
L1_wrapper()

,blue,green,square,circle
blue square,0.5,0.0,0.5,0.0
blue circle,0.5,0.0,0.0,1.0
green square,0.0,1.0,0.5,0.0


## Pragmatic speaker

Returns a distribution over utterances for a given object.

In [226]:
def utility(L1_probs):
    return L1_probs.apply(lambda x:np.exp(alpha * np.log(x) - cost(x.name)))

In [227]:
def speaker():
    L1 = L1_wrapper()
    return utility(L1).apply(lambda x:x / np.sum(x), axis=1)

## Pragmatic listener
Returns a distribution over objects for a given utterance. Near-ish human behavior.

In [243]:
def pragmatic_listener():
    S1 = speaker()
    return S1.transpose().apply(lambda x: x * uniform_probs() / np.sum(x * uniform_probs()), axis=1)

In [244]:
pragmatic_listener()

/home/gunnar/.local/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,blue square,blue circle,green square
blue,0.6,0.4,0.0
green,0.0,0.0,1.0
square,0.6,0.0,0.4
circle,0.0,1.0,0.0
